# Demo for the project(Genetic Algorithm implementation)

## background
- **Problem Context**: The project's goal is to develop a product allocation algorithm. specifically, allocating one type of product from multiple warehouses to multiple orders.  The input to this algorithm consists of the transport time matrix X between warehouses and orders (per unit), an array Y specifying the demand of each order, and an array Z detailing the current stock of the product in each warehouse. The output should be a matrix A indicating the volume of product transported from each warehouse to each order, and the minimize the maximum transportation time, i.e. elements in  Hadamard product fo A and X.

- **Known Constraints**: Sum of allocations for each order≥Demand, Sum of allocations for each warehouse≤Inventory; the objective is to minimize the maximum transportation time across all warehouse-order allocation pairs.
- **Technical Environment**: operate within a modified Jupyter Notebook environment.Python Version: 3.8




### Convert JSON Data to Numpy Arrays

The function `json_to_matrices` converts the input JSON data into a set of NumPy arrays that represent different variables in a logistics distribution problem.

#### Output:
- `A1`: 3D array representing unit transportation times. Dimensions: `(number of orders, number of warehouses, number of goods)`.
- `A2`: 2D array representing demand for each type of good for each order. Dimensions: `(number of orders, number of goods)`.
- `A3`: 2D array representing the stock of each type of good in each warehouse. Dimensions: `(number of warehouses, number of goods)`.
- `W1`: 1D array representing the priority of each order.
- `W2`: 1D array representing the priority of each warehouse.
- `order_list`: List of all orders in the JSON data.
- `warehouse_list`: List of all unique warehouse names in the JSON data.
- `goods_list`: List of all unique goods in the JSON data.
- `goods_dict`: Dictionary mapping each good to its unit size.

#### Workflow:
1. Parses the number of orders (`m`), number of unique goods (`k`), and number of unique warehouses (`n`).
2. Initializes matrices `A1`, `A2`, `A3` and arrays `W1`, `W2`.
3. Populates these matrices and arrays based on the JSON data.
4. Returns these as well as additional information like the order, warehouse, and goods lists.

#### Additional Notes:
- It uses NumPy for array operations.
- Make sure the JSON data is well-structured and adheres to the expected schema for proper functioning.

Run this function to convert the JSON data into usable matrices for the `logistics_distribution` algorithm.

In [5]:
import numpy as np
import json

def json_to_matrices(json_data):
    m = len(json_data['spdd'])  # number of orders

    # Create a dictionary to map goods to units
    goods_dict = {}
    for warehouse in json_data['ck']:
        warehouse_id = list(warehouse.keys())[0]
        for good in warehouse[warehouse_id]:
            if good['spnm'] not in goods_dict:
                goods_dict[good['spnm']] = good['lg']
    
    # Create a list of warehouses based on their order in 'ckdata'
    all_warehouses = [data['cknm'] for order in json_data['spdd'] for data in order['ckdata']]
    
    # Removing duplicates while preserving the order
    all_warehouses = list(dict.fromkeys(all_warehouses))

    all_goods_spdd = [order['spnm'] for order in json_data['spdd']]
    all_goods_ck = list(goods_dict.keys())
    all_goods = sorted(list(set(all_goods_spdd + all_goods_ck)))
    k = len(all_goods)  # Number of unique goods
    n = len(all_warehouses)  # number of warehouses
    
    # Initialize matrices
    A1 = np.zeros((m, n, k))
    A2 = np.zeros((m, k))
    A3 = np.zeros((n, k))
    # W1 = 1 / np.arange(1, m+1)  # We consider the priority of orders to be inversely proportional to their order
    # W2 = 1 / np.arange(1, n+1)  # We consider the priority of warehouses to be inversely proportional to their order
    W1 = np.arange(m, 0, -1) / m * 0.3 + 0.7  # We consider the priority of orders to decrease by a fixed interval
    W2 = np.arange(n, 0, -1) / n * 0.3 + 0.7  # We consider the priority of warehouses to decrease by a fixed interval
   
    for i, order in enumerate(json_data['spdd']):
        good_id = all_goods.index(order['spnm'])
        A2[i, good_id] = order['sl']
        for j, ckdata in enumerate(order['ckdata']):
            A1[i, j, good_id] = ckdata['dwyssj']
    
    for warehouse in json_data['ck']:
        warehouse_id = list(warehouse.keys())[0]
        if warehouse_id in all_warehouses:
            i = all_warehouses.index(warehouse_id)
            for good in warehouse[warehouse_id]:
                good_id = all_goods.index(good['spnm'])
                A3[i, good_id] = good['sl']

    order_list = json_data['spdd']
    warehouse_list = all_warehouses
    goods_list = all_goods
    
    return A1, A2, A3, W1, W2, order_list, warehouse_list, goods_list, goods_dict




### Function: `greedy_solution`

**Objective**: 

Develop an initializing distribution plan that maximizes overall satisfaction and minimizes maximum transportation time using a greedy algorithm.

**Inputs**: 

- `X`: A 3D numpy array of shape (m, n, k). It represents the unit transportation time.
- `Y`: A 2D numpy array of shape (m, k). It represents the demand for each type of good for each order.
- `Z`: A 2D numpy array of shape (n, k). It represents the stock of each type of good in each warehouse.
- `O`: A 1D numpy array of length m. It represents the priority of each order.
- `W`: A 1D numpy array of length n. It represents the priority of each warehouse.

**Output**: 

- A 3D numpy array (distribution plan) that maximizes overall satisfaction and minimizes maximum transportation time greedily.

**Implementation Details**:

The algorithm works by iterating through orders, goods, and warehouses. For each order and good type, it tries to allocate the good from the highest-priority warehouse. The quantity allocated is the minimum of the demand and the stock available in the warehouse. The demand and stock are adjusted with every allocation until the demand for that order and good type is satisfied or there are no more goods of that type in any warehouse.

---



In [6]:
def greedy_solution(X, Y, Z, O, W):
    """
    This function takes in five matrices X, Y, Z, O, W which represent
    different aspects of a logistics problem. The function should output a
    distribution plan to maximize overall satisfaction and minimize maximum
    transportation time greedily.

    Input:
    X: A 3D numpy array of shape (m, n, k) representing the unit transportation time
    Y: A 2D numpy array of shape (m, k) representing the demand for each type of good for each order
    Z: A 2D numpy array of shape (n, k) representing the stock of each type of good in each warehouse
    O: A 1D numpy array of length m representing the priority of each order
    W: A 1D numpy array of length n representing the priority of each warehouse

    Output:
    A distribution plan that maximizes overall satisfaction and minimizes maximum transportation time greedily.
    """
    Y = Y.copy()
    Z = Z.copy()
    A_greed = np.zeros_like(X)
    for i in range(A_greed.shape[0]):
        for k in range(A_greed.shape[2]):
            for j in range(A_greed.shape[1]):
                sati_ijk = np.minimum(Y[i,k], Z[j, k])
                A_greed[i,j,k] = sati_ijk
                Y[i,k] -= sati_ijk
                Z[j,k] -= sati_ijk
                if Y[i,k] <= 0:
                    break
            if Y[i,k] <= 0:
                break
    #print(np.sum(A_greed), A_greed)
    return A_greed

### Logistics Distribution Algorithm

The function `logistics_distribution` is designed to solve a logistics optimization problem. It takes into account various parameters such as unit transportation time, demand, stock, priority of orders, and priority of warehouses. The objective is to maximize overall satisfaction and minimize maximum transportation time.

#### Parameters:
- **X**: 3D numpy array (m, n, k), unit transportation time between orders and warehouses for each type of goods.
- **Y**: 2D numpy array (m, k), demand for each type of good for each order.
- **Z**: 2D numpy array (n, k), stock of each type of good in each warehouse.
- **O**: 1D numpy array (m), priority of each order.
- **W**: 1D numpy array (n), priority of each warehouse.

#### Output:
- Returns a JSON formatted string containing a distribution plan. The plan is a 3D numpy array (m, n, k).

#### Algorithm:
1. Use a greedy solution as an initial feasible solution.
2. Utilize a Genetic Algorithm (GA) to evolve better solutions over time.
3. Constrain the GA with penalties for unfeasible solutions and prioritize solutions that maximize overall satisfaction and minimize transportation time.

#### Additional Data Structures:
- `order_list`: List of dictionaries, each containing information about an order.
- `warehouse_list`: List of dictionaries, each containing information about a warehouse.
- `goods_list`: List of dictionaries, each containing information about a type of goods.
- `goods_dict`: Dictionary mapping the name of goods to additional details.

This function makes use of the `pygad` library for genetic algorithm implementation. It sets various GA parameters and finally runs the GA to find an optimized distribution plan.

In [7]:
import numpy as np
import pygad
import json

def logistics_distribution(X, Y, Z, O, W, order_list, warehouse_list, goods_list, goods_dict):
    """
    This function takes in five matrices X, Y, Z, O, W which represent 
    different aspects of a logistics problem. The function should output a 
    distribution plan to maximize overall satisfaction and minimize maximum 
    transportation time.
    
    Input:
    X: A 3D numpy array of shape (m, n, k) representing the unit transportation time
    Y: A 2D numpy array of shape (m, k) representing the demand for each type of good for each order
    Z: A 2D numpy array of shape (n, k) representing the stock of each type of good in each warehouse
    O: A 1D numpy array of length m representing the priority of each order
    W: A 1D numpy array of length n representing the priority of each warehouse
    
    Output:
    A distribution plan that maximizes overall satisfaction and minimizes maximum transportation time.
    A: A 3D numpy array of shape (m, n, k) representing the distribution plan
    """

    A_greed = greedy_solution(X,Y,Z,O,W)
    # 设置满足度和运输时间的权重
    alpha = 0.5
    beta = 0.5
    penalty_overall=0.1
    penalty_constrain = 1
    penalty_stock=0.1
    order_priority_scaling_factor=0.1
    # Normalize X and store the min and max for later denormalization
    X_min, X_max = X.min(), X.max()
    X_norm = (X - X_min) / (X_max - X_min+1e-7)

    # 初始化解向量 X
    A = np.zeros_like(X, dtype=int)
    
    # 处理订单需求数据
    Y_copy = Y.copy()
    Y_copy[np.where(Y==0)] = np.inf
    

    # 定义带有惩罚项的满足度计算函数
    def fitness_func_penalty(ga_instance, A, A_idx):
        # A is solution
        # setting constraints
        A.resize(X.shape)
        penalty = 0
        

        # adjust the penalties based on your requirements
        if (np.sum(A, axis=0) < 0).any():
            penalty += penalty_constrain * np.sum(
                np.abs(np.sum(A, axis=0)[np.sum(A, axis=0) < 0]))
        if ((np.sum(A, axis=0) - Z) > 0).any():
            penalty += penalty_constrain * np.sum(
                np.abs(np.sum(A, axis=0) - Z)[(np.sum(A, axis=0) - Z) > 0])
        if (np.sum(A, axis=1) < 0).any():
            penalty += penalty_constrain * np.sum(
                np.abs(np.sum(A, axis=1)[np.sum(A, axis=1) < 0]))
        if ((np.sum(A, axis=1) - Y) > 0).any():
            penalty += penalty_constrain * np.sum(
                np.abs(np.sum(A, axis=1) - Y)[(np.sum(A, axis=1) - Y) > 0])
        #if not sufficiency_flag[0]:
        if (Z - np.sum(A, axis=0) > 0).any():
            penalty +=  penalty_stock*np.sum(
                np.abs(Z - np.sum(A, axis=0))[Z - np.sum(A, axis=0) > 0])
        #if np.sum(A) - np.sum(Y) < 0:
        #    penalty +=   (np.sum(Y) - np.sum(A))/np.sum(Y) + 1
        penalty = penalty_overall * penalty

        B = A * X_norm
        B_norm = B / (np.minimum(np.max(Y), np.max(Z)) + 1e-10)
        O_scaled = O ** order_priority_scaling_factor 
        fitness_sati = alpha * np.sum(O_scaled * np.sum(
            np.sum(A * W[np.newaxis, :, np.newaxis], 
                   axis=1) / Y_copy, axis=-1),
                       axis=0) / (Y.shape[0]*Y.shape[1])
        fitness_time = beta * np.max(np.sum(B_norm, axis=(1, 2)) / (Z.shape[0]*Z.shape[1]))
        fitness = fitness_sati - fitness_time - penalty

        return fitness

    # 

    # 设置使用带有惩罚项的满足度计算函数
    fitness_function = fitness_func_penalty

    # 设置遗传算法的参数
    num_generations = 50000         # 迭代次数
    num_parents_mating = 8         # 交配的父母数量
    sol_per_pop = 16                # 种群中的解的数量
    num_genes = A.shape[0]*A.shape[1]*A.shape[2]  # 每个解中的基因数量
    # init_range_low = 0            # 初始化解的下限
    # init_range_high = 5            # 初始化解的上限
    parent_selection_type = "sss"  # 选择父母的类型
    keep_parents = 2              # 保留父母的数量
    crossover_type = "single_point" # 交叉类型
    mutation_type = "random"       # 变异类型
    mutation_percent_genes = 30    # 变异基因的百分比
    # 计算基因空间的上限
    gene_space_high = np.minimum(Y[:,np.newaxis,:], Z[np.newaxis,:,:]).flatten().tolist()
    gene_space = [np.arange(0, int(high+1)) for high in gene_space_high]
    # 设置初始化种群，其中包含一个贪心可行解
    initial_population = np.random.randint(0, np.max(gene_space_high), (sol_per_pop, num_genes))
    initial_population[0] = A_greed.flatten()

    # 创建遗传算法实例
    ga_instance = pygad.GA(
        num_generations=num_generations,  # 总共的进化代数
        num_parents_mating=num_parents_mating,  # 每一代中进行交配的父代数量
        fitness_func=fitness_function,  # 评价函数，用于评价每个候选解的适应度
        sol_per_pop=sol_per_pop,  # 种群中的个体数量
        num_genes=num_genes,  # 每个个体中的基因数量
        initial_population=initial_population, # 初始化种群
        parent_selection_type=parent_selection_type,  # 父代选择策略
        keep_parents=keep_parents,  # 下一代中保持的父代数量
        crossover_type=crossover_type,  # 交叉类型
        mutation_type=mutation_type,  # 变异类型
        mutation_percent_genes=mutation_percent_genes,  # 需要变异的基因的百分比
        gene_type=int,  # 基因的数据类型
        gene_space=gene_space,  # 可选的基因值范围
        stop_criteria=["saturate_2000"]  # 停止进化的条件
    )


    ga_instance.run()

    print("Number of generations passed is {generations_completed}".format(generations_completed= ga_instance.generations_completed))
    solution, solution_fitness, solution_idx = ga_instance.best_solution()

    solution.resize(X.shape)
    A = solution

    print("A: best solution : {solution}".format(solution=solution))
    print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
    
    # Prepare the data for the JSON output
    # Prepare the data for the JSON output
    data = []
    for m_index, m in enumerate(order_list):
        for n_index, n in enumerate(m['ckdata']):
            for k_index, k in enumerate(goods_list):
                if k == m['spnm']:  # We only need to add entries for the goods in the order
                    quantity = int(A[m_index][n_index][k_index])
                    dispatch_time = float(X[m_index][n_index][k_index] * A[m_index][n_index][k_index])
                    
                    # Do not append to data if quantity and dispatch time are 0
                    if quantity != 0 or dispatch_time != 0.0:
                        data.append({
                            "ddnm": m['ddnm'],  # order code
                            "cknm": n['cknm'],  # warehouse code
                            "qynm": m['qynm'],  # enterprise code
                            "spnm": k,  # goods code
                            "sl": quantity,  # quantity, convert numpy int64 to native Python int
                            "lg": goods_dict[m['spnm']] ,  # dimension not given in the data on 0705
                            "dpsj": dispatch_time,  # dispatch time, convert numpy float64 to native Python float if necessary
                        })

    # Package everything into a dict, ready for conversion to JSON
    result = {
        "code": 200,
        "data": data,
    }

    return json.dumps(result,ensure_ascii=False, indent=4)


### Test Function for Logistics Distribution Algorithm

The function `test_function` is a driver function for testing the `logistics_distribution` algorithm. 

#### Workflow:
1. Read the JSON data from a file, which contains all the necessary information like unit transportation times, demand for each type of good for each order, stock in each warehouse, priorities, etc.
2. Convert this JSON data into matrices and arrays using the utility function `json_to_matrices`.
3. Print these matrices for verification.
4. Run the `logistics_distribution` algorithm with the parsed data.
5. Print and save the result as a JSON file in the `test` directory.

#### Additional Notes:
- This function uses the `numpy` library for array manipulations.
- It uses `json` library to handle JSON data.
- It relies on a utility function `json_to_matrices` to convert JSON data into numpy arrays.
- It saves the result in the 'test/result.json' file for future reference. 

Run this function to test the `logistics_distribution` algorithm using data from `data.json`. Make sure that the utility function `json_to_matrices` and the `logistics_distribution` function are properly imported from their respective modules.

In [8]:
import numpy as np
import json

def test_function():
    with open("data.json",encoding='utf-8') as f:
        json_data = json.load(f)

    A1, A2, A3, W1, W2, order_list, warehouse_list, goods_list,goods_dict = json_to_matrices(json_data)

    for i in range(A1.shape[0]):
        for j in range(A1.shape[1]):
            print(f"For order {i+1}, warehouse {j+1}, the unit transportation times are {A1[i, j]} for goods {list(range(1, A1.shape[2]+1))}")
    
    print("A2: Demand for each type of good for each order. Each row corresponds to an order, each column corresponds to a good.")
    print(A2)

    print("A3: Stock of each type of good in each warehouse. Each row corresponds to a warehouse, each column corresponds to a good.")
    print(A3)

    print("W1: Priority of each order. The lower the index, the higher the priority.")
    print(W1)

    print("W2: Priority of each warehouse. The lower the index, the higher the priority.")
    print(W2)
    result = logistics_distribution(A1, A2, A3, W1, W2, order_list, warehouse_list, goods_list,goods_dict)
    #logistics_distribution(A1, A2, A3, W1, W2)
    print(result)
    with open('test/result.json', 'w',encoding='utf-8') as f:
        json.dump(result, f)
    
if __name__ == "__main__":
    test_function()

For order 1, warehouse 1, the unit transportation times are [60.] for goods [1]
For order 1, warehouse 2, the unit transportation times are [90.] for goods [1]
For order 1, warehouse 3, the unit transportation times are [210.] for goods [1]
For order 1, warehouse 4, the unit transportation times are [240.] for goods [1]
For order 2, warehouse 1, the unit transportation times are [6.] for goods [1]
For order 2, warehouse 2, the unit transportation times are [9.] for goods [1]
For order 2, warehouse 3, the unit transportation times are [21.] for goods [1]
For order 2, warehouse 4, the unit transportation times are [24.] for goods [1]
For order 3, warehouse 1, the unit transportation times are [6.] for goods [1]
For order 3, warehouse 2, the unit transportation times are [9.] for goods [1]
For order 3, warehouse 3, the unit transportation times are [21.] for goods [1]
For order 3, warehouse 4, the unit transportation times are [24.] for goods [1]
A2: Demand for each type of good for each 